# Data selection to reduce the original GovBR corpus

In [1]:
import pandas as pd
import numpy as np
import re
import unidecode
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

#### Read data:

In [2]:
folha = pd.read_csv("../data/govbr/articles_folha/folhasp.csv")
df = pd.read_csv("../data/govbr/GovBR_users_corpus.csv", sep=';')

In [3]:
df = df.dropna()

In [22]:
folha

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...
...,...,...,...,...,...,...
167048,"Em cenário de crise, tucano Beto Richa assume ...",O tucano Beto Richa tinha tudo para começar se...,2015-01-01,poder,NaN,http://www1.folha.uol.com.br/poder/2015/01/156...
167049,Filho supera senador Renan Calheiros e assume ...,O economista Renan Filho (PMDB) assume nesta q...,2015-01-01,poder,NaN,http://www1.folha.uol.com.br/poder/2015/01/156...
167050,"Hoje na TV: Tottenham x Chelsea, Campeonato In...",Destaques da programação desta quinta-feira (1...,2015-01-01,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/01/1...
167051,Kim Jong-un diz estar aberto a se reunir com p...,"O líder norte-coreano, Kim Jong-un, disse nest...",2015-01-01,mundo,NaN,http://www1.folha.uol.com.br/mundo/2015/01/156...


#### Filter folha corpus to the Politics content:

In [4]:
def combine(title, text):
    return ' '.join(title, text)

In [21]:
folha['category'].value_counts().index

Index(['poder', 'colunas', 'mercado', 'esporte', 'mundo', 'cotidiano',
       'ilustrada', 'opiniao', 'paineldoleitor', 'saopaulo', 'tec', 'tv',
       'educacao', 'turismo', 'ilustrissima', 'ciencia', 'equilibrioesaude',
       'sobretudo', 'bbc', 'folhinha', 'empreendedorsocial', 'comida',
       'asmais', 'ambiente', 'seminariosfolha', 'serafina',
       'o-melhor-de-sao-paulo', 'vice', 'guia-de-livros-discos-filmes',
       'topofmind', 'banco-de-dados', 'dw', 'especial', 'cenarios-2017',
       'infograficos', 'rfi', 'guia-de-livros-filmes-discos', 'multimidia',
       'treinamento', 'treinamentocienciaesaude', 'mulher', 'euronews',
       'ombudsman', 'contas-de-casa', 'musica', 'bichos', '2016', '2015'],
      dtype='object')

In [24]:
corpus = folha.loc[
    (folha['category'] == 'poder') | \
    (folha['category'] == 'mercado') | \
    (folha['category'] == 'mundo') | \
    (folha['category'] == 'opiniao') | \
    (folha['category'] == 'dw') | \
    (folha['category'] == 'euronews')][['title', 'text']]

corpus['content'] = corpus['title'] + ' ' + corpus['text']

corpus

,title,text,content
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,"Lula diz que está 'lascado', mas que ainda tem..."
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,Três reportagens da Folha ganham Prêmio Petrob...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",CBSS inicia acordos com fintechs e quer 30% do...
5,"Em encontro, Bono pergunta a Macri sobre argen...","O vocalista da banda irlandesa U2, Bono, fez u...","Em encontro, Bono pergunta a Macri sobre argen..."
6,"Posso sair do Brasil quando e como quiser, diz...",O italiano Cesare Battisti disse nesta segunda...,"Posso sair do Brasil quando e como quiser, diz..."
...,...,...,...
167045,Governador do RS inicia 2015 com quebra-cabeça...,O peemedebista José Ivo Sartori toma posse nes...,Governador do RS inicia 2015 com quebra-cabeça...
167046,"Apoiado pelos irmãos Gomes, petista toma posse...",Engenheiro agrônomo e servidor licenciado do I...,"Apoiado pelos irmãos Gomes, petista toma posse..."
167048,"Em cenário de crise, tucano Beto Richa assume ...",O tucano Beto Richa tinha tudo para começar se...,"Em cenário de crise, tucano Beto Richa assume ..."
167049,Filho supera senador Renan Calheiros e assume ...,O economista Renan Filho (PMDB) assume nesta q...,Filho supera senador Renan Calheiros e assume ...


#### Generating the tf-idf for political related words:

In [25]:
nltk.download('stopwords')
stopwords = map(
    lambda _ : unidecode.unidecode(_),
    nltk.corpus.stopwords.words('portuguese')
)
stopwords = set(stopwords)

def cleaning(content):
    # Lower string
    new_content = content.lower()

    # Remove diacritics. Ex: replace ç -> c
    new_content = unidecode.unidecode(new_content)

    # Get only words and return
    return re.findall('([a-zA-Z]+)', new_content)


vec = TfidfVectorizer(
    tokenizer=cleaning,
    stop_words=stopwords,
)

X = vec.fit_transform([' '.join(corpus['content'])])

X

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\samuel.caetano\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<1x165381 sparse matrix of type '<class 'numpy.float64'>'
	with 165381 stored elements in Compressed Sparse Row format>

In [26]:
vec.get_feature_names_out().shape

(165381,)

#### Organize selected tokens:

In [27]:
tfidf = pd.DataFrame(
    np.sum(X, axis=0),
    columns=vec.get_feature_names_out().tolist(),
)

tfidf = tfidf.T

tfidf

,0
C,0.000002
EUR,0.004216
PS,0.000469
SS,0.000093
aa,0.000088
...,...
zypora,0.000002
zypries,0.000007
zysk,0.000005
zzyym,0.000002


In [28]:
proxy = tfidf[tfidf >= tfidf.mean()].dropna().index

proxy

Index(['EUR', 'PS', 'ab', 'abaaoud', 'abadi', 'abaixo', 'abalada', 'abalo',
       'abalou', 'abandona',
       ...
       'zelotes', 'zerar', 'zero', 'zika', 'zona', 'zonas', 'zoologico',
       'zuckerberg', 'zveiter', 'zwi'],
      dtype='object', length=14378)

#### Filter text from GovBR:

In [29]:
df

,Tweet_Id,User_Seq,Class,Created_At,Tweet_Type,Word_Count,Mentions,Text
0,1208848090704359424,1,for,Sun Dec 22 21:38:04 +0000 2019,Tweet,9,0,"Não vou rir, pois é errado zombar dos doentes."
1,1208909048428539904,1,for,Sun Dec 22 21:42:03 +0000 2019,Tweet,3,0,Seguindo imediatamente! 👍🇧🇷
2,1208852119828217856,1,for,Sun Dec 22 21:43:23 +0000 2019,Tweet,4,0,Esqueci desse detalhe KKKK
3,1208731700014067712,1,for,Sun Dec 22 22:02:36 +0000 2019,Tweet,3,0,Verdade! 👍
4,1208764004593754112,1,for,Sun Dec 22 22:05:16 +0000 2019,Tweet,6,0,"Muito rabo preso, eu não acredito!"
...,...,...,...,...,...,...,...,...
13540706,1290411228070916096,7155,against,Mon Aug 03 20:20:16 +0000 2020,Tweet,8,0,FOI PRESO POBRE... MAIOR ESTADISTA PORTUGUÊS !...
13540707,1290429064474513409,7155,against,Mon Aug 03 20:27:18 +0000 2020,Tweet,1,0,Ｆｅｃｈａｒ ｏｓ ｂａｎｃｏｓ.mp3
13540708,1290468731676700674,7155,against,Tue Aug 04 09:35:05 +0000 2020,Tweet,6,0,"Man, o flex do [papel], jesus."
13540709,1290724933241274368,7155,against,Tue Aug 04 16:02:59 +0000 2020,Reply,24,0,Take que absolutamente ninguém pediu: pôr quem...


In [30]:
proxy

Index(['EUR', 'PS', 'ab', 'abaaoud', 'abadi', 'abaixo', 'abalada', 'abalo',
       'abalou', 'abandona',
       ...
       'zelotes', 'zerar', 'zero', 'zika', 'zona', 'zonas', 'zoologico',
       'zuckerberg', 'zveiter', 'zwi'],
      dtype='object', length=14378)

In [31]:
from tqdm import tqdm_notebook

In [32]:
idxs = []
for idx, row in tqdm_notebook(df.dropna().iterrows(), total=df.shape[0], desc="Processing records"):
    
    text = set(cleaning(row['Text']))

    # Token appearance
    ta = sum([1 for token in text if token in proxy])

    try:
        score = ta / len(text)
    except ZeroDivisionError:
        score = 0
    
    if score > 0.25 and score < 1:
        idxs.append(idx)
        # print(idx, df.shape[0])# text, score, ta, len(text), )

c:\Users\samuel.caetano\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


Processing records:   0%|          | 0/13540676 [00:00<?, ?it/s]

In [35]:
idxs

[0,
 2,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 15,
 16,
 17,
 19,
 20,
 21,
 24,
 25,
 27,
 29,
 30,
 31,
 32,
 34,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 45,
 46,
 48,
 49,
 51,
 52,
 53,
 54,
 55,
 56,
 59,
 60,
 61,
 62,
 67,
 70,
 71,
 74,
 75,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 87,
 88,
 89,
 91,
 92,
 93,
 96,
 97,
 99,
 101,
 103,
 105,
 106,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 118,
 119,
 120,
 123,
 124,
 125,
 126,
 127,
 129,
 131,
 132,
 133,
 135,
 139,
 141,
 142,
 144,
 146,
 147,
 148,
 149,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 160,
 161,
 162,
 163,
 164,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 175,
 176,
 179,
 180,
 181,
 182,
 184,
 185,
 186,
 187,
 188,
 189,
 192,
 198,
 201,
 202,
 203,
 204,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 217,
 219,
 220,
 221,
 222,
 225,
 226,
 227,
 228,
 229,
 231,
 232,
 233,
 234,
 235,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 25

In [38]:
data = df[df.index.isin(idxs)].sample(4010)

data

,Tweet_Id,User_Seq,Class,Created_At,Tweet_Type,Word_Count,Mentions,Text
2345976,1276159970334126081,1357,for,Thu Jun 25 11:32:12 +0000 2020,Tweet,4,0,"Sem Lacombe, sem audiência!"
12787252,1255442152504594434,6807,against,Wed Apr 29 08:34:46 +0000 2020,Tweet,19,0,Gente esse senhor é tão esquesito. Parece semp...
12544360,1177621141386604548,6684,against,Fri Sep 27 13:28:53 +0000 2019,Tweet,5,0,Rumo a série B. …
9233525,1250155008118718466,5150,against,Tue Apr 14 17:33:06 +0000 2020,Reply,11,0,Finalmente tendo tempo para ver velhos álbuns ...
13502022,1201677274086592512,7140,against,Mon Dec 02 22:39:22 +0000 2019,Tweet,13,0,Aqui está uma música para você… Partners In Cr...
...,...,...,...,...,...,...,...,...
2214566,1290031513183166468,1285,for,Mon Aug 03 00:35:40 +0000 2020,Tweet,41,0,"Não é ideologia, é desconfiança dessa fixação ..."
7333614,1242589642349850627,4263,against,Tue Mar 24 20:10:31 +0000 2020,Tweet,29,0,"Com a demanda, o medo é piorar a qualidade dos..."
288117,71997041621999617,169,for,Sat May 21 14:53:27 +0000 2011,Tweet,11,0,"FRESNOnoSWU ??? por favor, cês tão querendo ""R..."
6726360,701799501779116036,3961,against,Mon Feb 22 13:03:34 +0000 2016,Tweet,21,0,Chamaram o Roger de coxinha no show dos Stones...


In [39]:
data['Class'].value_counts()

for        2015
against    1995
Name: Class, dtype: int64

#### Save selected to file:

In [40]:
data.to_csv('../data/govbr/GovBR.csv', index=False)

#### Split selected data between train and test:

In [41]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['Text'], data['Class'], test_size=0.2)

X_train.shape, X_test.shape

((3208,), (802,))

In [52]:
train = pd.DataFrame(
    {
    'Text': X_train,
    'Class': y_train,
    }
)

test = pd.DataFrame(
    {
    'Text': X_test,
    'Class': y_test,
    }
)

In [53]:
train, test

(                                                       Text    Class
 11988523                                 Eita Deus bom esse  against
 10722119  Brasil supera média mundial e reduz 73% da mor...  against
 3957988   No dia em que milhares sairão às ruas em prol ...      for
 12322833  tá mas minha tia fez um bolo de aniversário pr...  against
 3348039   Fala mais...tem que cobrar cada centavo...inve...      for
 ...                                                     ...      ...
 10625544  Quando não se tem costas quentes como ser enge...  against
 927396                                 É agora ou nunca   …      for
 12088244  @  DEPUTADOS TIRAM A CLT DO POBRE MAS TEM 2 FÉ...  against
 9778672   A gestão de @ recuperou o sistema de esgotamen...  against
 2628774   Fonte     Ação pede que União obrigue China a ...      for
 
 [3208 rows x 2 columns],
                                                        Text    Class
 1980870                                  2 zero a esquerda.  

#### Convert 'for' label to 1, and 'against' to 0:

In [60]:
train['Class'].loc[train['Class'] == 'against'] = 0
train['Class'].loc[train['Class'] == 'for'] = 1

test['Class'] .loc[test['Class'] == 'against']= 0
test['Class'].loc[test['Class'] == 'for'] = 1

In [63]:
train.to_csv('../data/govbr/govbr.train.csv', index=False)
test.to_csv('../data/govbr/govbr.test.csv', index=False)